# Investigação da consistência dos dados

Ao iniciar uma análise das despesas da pandemia do COVID-19 reparamos que os dados
tinham alguns problemas. Algumas entradas estavam duplicadas e outras não batiam.

Essa análise visa comparar os dados apresentados no Tribunal de Contas dos Municípios da Bahia
(TCM-BA) e o Portal da Transparência da Prefeitura de Feira de Santana.

## Dados utilizados

Para avaliar os dados escolhemos o mês de Julho de 2020 (escolhemos aleatoriamente mas poderia ser qualquer um)
e selecionamos os **pagamentos** da **Prefeitura Municipal de Feira de Santana**.

* [Portal da Transparência da Prefeitura de Feira de Santana](http://www.transparencia.feiradesantana.ba.gov.br/index.php?view=despesa)
* [Tribunal de Contas dos Municípios da Bahia](https://www.tcm.ba.gov.br/consulta-de-despesas)

Exportamos esses dados para XLS e CSV, respectivamente.

## Vamos a análise!

In [65]:
import pandas as pd


cityhall_df = pd.read_excel("../prefeitura-despesas-prefeitura-julho-2020.csv.xls")
tcm_df = pd.read_json("../tcmba-despesas-prefeitura-julho-2020.json")

Uma amostra dos dados da prefeitura:

In [66]:
cityhall_df.sample(3)

,Data de Publicacao,Fase,Numero,N do processo,Bem / Servico Prestado,Credor,CPF / CNPJ,Valor,Funcao,Subfuncao,Natureza,Fonte,Modalidade
617,23/07/2020,PAGAMENTO,08785-20,237-2019,REF A DESPESA COM SERVIÇO DE PUBLICIDADE E PRO...,A.C.PROPAGANDA LTDA. ...,02.811.243/0001-35,"596,08",24 - COMUNICACOES ...,131 - COMUNICACAO SOCIAL ...,339039010000 - Publicidades e Propaganda ...,0000 - RECURSOS ORDINARIOS ...,CONCORRENCIA
788,23/07/2020,PAGAMENTO,6509,,REFERENTE A VIAGEM A SALVADOR PARA CONDUZIR PA...,HUGO MARCONDES DE A. SANTANA ...,008.080.355-50,"100,00",10 - SAUDE ...,122 - ADMINISTRACAO GERAL ...,339014000000000000 - Diarias-Civil ...,0002 - REC.IMP.TRANSF.IMP.SAUDE 15 ...,Isento
2446,06/07/2020,PAGAMENTO,5896,,REFERENTE FOLHA DE PAGAMENTO AUXILIO DOENCA EN...,FOLHA PAG. ENDEMIAS ESTATUTARI ...,08.576.590/0001-07,"39,20",10 - SAUDE ...,305 - VIGILIANCIA EPIDEMIOLOGICA ...,319011010000000000 - V.Vant.Fixas P.CivilVe.Ba...,0002 - REC.IMP.TRANSF.IMP.SAUDE 15 ...,Isento


In [67]:
cityhall_df.shape

(2721, 13)

Uma amostra dos dados do TCM-BA:

In [68]:
tcm_df.sample(3)

,NumeroDocumeto,empenho,data,municipio,unidade,cd_Unidade,orgao,unidadeOrcamentaria,credor,valor,cd_Orgao,cd_UnidadeOrcamentaria,dataEmpenho
830,06134-20-FMS,20000257,2020-07-10 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,BIOCENTER CENT HEM.E PAT.CLI.L,16536.39,11,1111,2020-01-02 00:00:00.000
1178,08542-20-PMFS,20000015,2020-07-16 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SEC.DE COMUNICACAO SOCIAL,SECRETARIA DE COMUNIC.SOCIAL,MERCADO DE PRO.E MARK. LTDA,1980.00,7,707,2020-02-17 00:00:00.000
561,06074-20-FMS,20001403,2020-07-08 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,IMAPS-INST.MARIE PIERRE DE SAUDE ASSOC. PROT. MAT,43790.38,11,1111,2020-03-03 00:00:00.000


In [69]:
tcm_df.shape

(2143, 13)

A partir do número de linhas podemos ver que os números não batem.
Mas vamos tentar cruzar os códigos de empenho para ver o que encontramos.

Antes de começarmos a explorar um pouco mais, vamos mapear os campos de cada arquivo:

| Campo (o que é) | Prefeitura | TCM-BA        | Observações |
| ------------- | ------------- | ------------- | ------------- |
| Data | `Data de Publicacao` | `data` | Assumindo que essa é a data do pagamento e não a data do empenho |
| Fase da despesa | `Fase` |  | |
| Número da despesa | `Numero` | `empenho` | |
| Número do processo | `N do processo` | `NumeroDocumeto` | Necessário confirmar |
| Descrição da despesa | `Bem / Servico Prestado` |  | |
| Credor | `Credor` | `credor` | |
| CPF/CNPJ do credor | `CPF / CNPJ` | | Presente apenas na página de detalhes |
| Valor | `Valor` | `valor` | |
| Função | `Funcao` | | |
| Subfunção | `Subfuncao` | | |
| Natureza | `Natureza` | | |
| Fonte | `Fonte` | | |
| Modalidade | `Modalidade` |  | |


## Match dos pagamentos

Infelizmente não temos tantos campos assim para fazer o _match_ das despesas.
Mas podemos tentar a partir do número da despesa e o número do processo.

Em alguns números das despesas nos dados da prefeitura não vem com o zero na frente
e nem com o ano da despesa. Por exemplo: `6848` na prefeitura e `06848-20-FMS` no TCM.
Pelo padrão, os números da despesa são formados de 5 dígitos, acrescidos do ano (`\d{5}-\d{2}-\w+`).
Podemos padronizar os que não atendem esse padrão em uma nova coluna. O problema é inferir o ano,
pois não temos como saber se todas as despesas foram apenas desse ano.

O número da despesa, representado por `NumeroDocumeto` no TCM-BA, vem seguido do órgão.
Exemplo: `07613-20-PMFS`. Precisamos cortar essa parte para bater com o número nos dados da prefeitura.

In [70]:
cityhall_df["Numero"]

0       09149-20                      
1       09150-20                      
2       3107                          
3       3106                          
4       3105                          
                     ...              
2716    08717-20                      
2717    08715-20                      
2718    08888-20                      
2719    08889-20                      
2720    09910-20                      
Name: Numero, Length: 2721, dtype: object

In [71]:
tcm_df["NumeroDocumeto"]

0        05705-20-FMS
1        05734-20-FMS
2        05716-20-FMS
3        05744-20-FMS
4        05714-20-FMS
            ...      
2138    09900-20-PMFS
2139    09901-20-PMFS
2140     06848-20-FMS
2141     06847-20-FMS
2142     06844-20-FMS
Name: NumeroDocumeto, Length: 2143, dtype: object

In [72]:
def split_number_from_agency(value):
    result = value["NumeroDocumeto"].rsplit("-", maxsplit=1)
    number = None
    agency = None
    if len(result) == 2:
        number = result[0]
        agency = result[1]
    if len(result) == 1:
        number = result[0]
    return pd.Series([number, agency],index=['numeroProcesso','Orgao'])


tcm_df[["numeroProcesso", "Orgao"]] = tcm_df.apply(split_number_from_agency, axis=1)
tcm_df.sample(5)

,NumeroDocumeto,empenho,data,municipio,unidade,cd_Unidade,orgao,unidadeOrcamentaria,credor,valor,cd_Orgao,cd_UnidadeOrcamentaria,dataEmpenho,numeroProcesso,Orgao
1058,08436-20-PMFS,20000001,2020-07-16 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SEC.DESENVOLVIMENTO ECONOMICO,SEC.M.TRAB.TUR.DESENV ECONOM,TELEMAR NORTE LESTE S/A,131.46,14,1414,2020-01-02 00:00:00.000,08436-20,PMFS
1797,06533-20-FMS,20002062,2020-07-24 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,ANTENOR FRANCISCO DA SILVA,50.00,11,1111,2020-06-05 00:00:00.000,06533-20,FMS
1293,09080-20-PMFS,20000686,2020-07-17 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SEC.DESENVOLVIMENTO SOCIAL,UNIDADE GESTORA DO FMAS,I.N.S.S. - FOLHA,2665.77,12,1224,2020-04-01 00:00:00.000,09080-20,PMFS
1056,06543-20-FMS,20000666,2020-07-15 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,TELEMAR NORTE LESTE S/A,140.89,11,1111,2020-01-02 00:00:00.000,06543-20,FMS
818,06138-20-FMS,20000281,2020-07-10 00:00:00.000,FEIRA DE SANTANA,Prefeitura Municipal de FEIRA DE SANTANA,129,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,HL DE OLIVEIRA NETO E CIA LTDA,78702.36,11,1111,2020-01-02 00:00:00.000,06138-20,FMS


Hora de verificar os encontrados! Como o número de despesas da prefeitura é maior do que
o número de despesas no TCM-BA vamos utilizar os dados da prefeitura como base.

In [113]:
import re


def normalize_process_number(value):
    # TODO converter para inteiro, para facilitar o merge
    # 06848/6848/06848-20-PMS para 6848
    if value:
        result = re.findall(r"\d+", str(value))
        if result:
            return int(result[0])

tcm_df["numero_tcmba"] = tcm_df["NumeroDocumeto"].apply(normalize_process_number)
cityhall_df["numero_prefeitura"] = cityhall_df["Numero"].apply(normalize_process_number)

In [109]:
merged_df = pd.merge(cityhall_df, tcm_df, how='inner', left_on=["numero_prefeitura"], right_on=["numero_tcmba"], suffixes=('_prefeitura', '_tcmba'))
merged_df

,Data de Publicacao,Fase,Numero,N do processo,Bem / Servico Prestado,Credor,CPF / CNPJ,Valor,Funcao,Subfuncao,...,orgao,unidadeOrcamentaria,credor,valor,cd_Orgao,cd_UnidadeOrcamentaria,dataEmpenho,numeroProcesso,Orgao,numero_tcmba
0,31/07/2020,PAGAMENTO,6847.0,45-2017-11D,REF. A DESPESAS COM TARIFAS BANCARIAS MES 07/2...,BANCO DO BRASIL S/A CENTRO ...,00.000.000/0041-89,949.75,10 - SAUDE ...,122 - ADMINISTRACAO GERAL ...,...,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,BANCO DO BRASIL S/A CENTRO,949.75,11,1111,2020-01-02 00:00:00.000,b'06847-20',FMS,6847
1,31/07/2020,PAGAMENTO,6844.0,,REF. A DESPESAS COM TARIFAS BANCARIAS MES 07/2...,CAIXA ECONOMICA FEDERAL - ...,00.360.305/0068-11,190.58,10 - SAUDE ...,304 - VIGILANCIA SANITARIA ...,...,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,CAIXA ECONOMICA FEDERAL -,190.58,11,1111,2020-01-02 00:00:00.000,b'06844-20',FMS,6844
2,31/07/2020,PAGAMENTO,6848.0,45-2017-11D,REF. A DESPESAS COM TARIFAS BANCARIAS MES 07/2...,BANCO DO BRASIL S/A CENTRO ...,00.000.000/0041-89,623.80,10 - SAUDE ...,304 - VIGILANCIA SANITARIA ...,...,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,BANCO DO BRASIL S/A CENTRO,623.80,11,1111,2020-01-02 00:00:00.000,b'06848-20',FMS,6848
3,31/07/2020,PAGAMENTO,9169.0,,REFERENTE A DESPESA COM BLOQUEIO JUDICIAL ...,TRIBUNAL R.DO TRAB.DA 5.REGIAO ...,02.011.574/0001-90,131788.71,28 - ENCARGOS ESPECIAIS ...,843 - SERVICO DA DIVIDA INTERNA ...,...,SECRETARIA DA FAZENDA,SECRETARIA DA FAZENDA,TRIBUNAL R.DO TRAB.DA 5.REGIAO,131788.71,6,606,2020-04-01 00:00:00.000,b'09169-20',PMFS,9169
4,31/07/2020,PAGAMENTO,9485.0,,REFERENTE A DESPESA COM TARIFA BANCARIA ...,BANCO DO BRASIL S/A CENTRO ...,00.000.000/0041-89,3497.26,04 - ADMINISTRACAO ...,122 - ADMINISTRACAO GERAL ...,...,SECRETARIA DA FAZENDA,SECRETARIA DA FAZENDA,BANCO DO BRASIL S/A CENTRO,3497.26,6,606,2020-07-01 00:00:00.000,b'09485-20',PMFS,9485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1833,01/07/2020,PAGAMENTO,5742.0,051-20161111,REF. PRESTACAO DE SERVICOS NO NASF NO PERIODO ...,REDESAUDE COOPERATIVA DE TRABALHO ...,01.599.748/0001-15,245271.74,10 - SAUDE ...,301 - ATENCAO BASICA ...,...,SECRETARIA DE SAUDE,SECRETARIA DE SAUDE,REDESAUDE COOPERATIVA DE TRABALHO,230964.23,11,1111,2020-05-01 00:00:00.000,b'05742-20',FMS,5742
1834,01/07/2020,PAGAMENTO,8717.0,,ADIANTAMENTO EMPENHO 20000062OPARA MARYNES ROD...,MARYNES RODRIGUES DE MIRANDA ...,14.043.574/0001-51,-656.00,04 - ADMINISTRACAO ...,122 - ADMINISTRACAO GERAL ...,...,SECRETARIA DA FAZENDA,SECRETARIA DA FAZENDA,MARYNES RODRIGUES DE MIRANDA,-656.00,6,606,2020-02-18 00:00:00.000,b'08717-20',PMFS,8717
1835,01/07/2020,PAGAMENTO,8715.0,,ADIANTAMENTO EMPENHO 20000001PPARA EDNALVA VIT...,EDNALVA VITORIO DE SOUZA ...,14.043.574/0001-51,-38.00,20 - AGRICULTURA ...,122 - ADMINISTRACAO GERAL ...,...,SECRETARIA DE AGRICULTURA,SECRETARIA DE AGRICULTURA,EDNALVA VITORIO DE SOUZA,-38.00,16,1616,2020-01-20 00:00:00.000,b'08715-20',PMFS,8715
1836,01/07/2020,PAGAMENTO,8888.0,,REFERENTE A DESPESA COM TARIFA BANCARIA ...,BANCO DO BRASIL S/A CENTRO ...,00.000.000/0041-89,5.50,08 - ASSISTENCIA SOCIAL ...,122 - ADMINISTRACAO GERAL ...,...,SEC.DESENVOLVIMENTO SOCIAL,SEC.DESENVOLVIMENTO SOCIAL,BANCO DO BRASIL S/A CENTRO,5.50,12,1212,2020-03-02 00:00:00.000,b'08888-20',PMFS,8888


Infelizmente não encontramos nada (devido a um bug no pandas). #FIXME

Vamos dar uma olhada em como estão os dados do número da despesa na prefeitura.

In [74]:
cityhall_df["Numero"].unique()

array(['09149-20                      ', '09150-20                      ',
       '3107                          ', ...,
       '08888-20                      ', '08889-20                      ',
       '09910-20                      '], dtype=object)

In [75]:
tcm_df["numeroProcesso"].unique()

array(['05705-20', '05734-20', '05716-20', ..., '06848-20', '06847-20',
       '06844-20'], dtype=object)

Vamos tentar pela data e pelo valor das despesas.

In [114]:
fields = [
    "Data de Publicacao", "data",
    "numero_tcmba", "numero_prefeitura",
    "Numero", "NumeroDocumeto",
    "N do processo", "numeroProcesso",
    "Credor", "credor",
    "Valor", "valor",
]

merged_df[fields]

,Data de Publicacao,data,numero_tcmba,numero_prefeitura,Numero,NumeroDocumeto,N do processo,numeroProcesso,Credor,credor,Valor,valor
0,31/07/2020,2020-07-31 00:00:00.000,6847,6847.0,6847.0,06847-20-FMS,45-2017-11D,b'06847-20',BANCO DO BRASIL S/A CENTRO ...,BANCO DO BRASIL S/A CENTRO,949.75,949.75
1,31/07/2020,2020-07-31 00:00:00.000,6844,6844.0,6844.0,06844-20-FMS,,b'06844-20',CAIXA ECONOMICA FEDERAL - ...,CAIXA ECONOMICA FEDERAL -,190.58,190.58
2,31/07/2020,2020-07-31 00:00:00.000,6848,6848.0,6848.0,06848-20-FMS,45-2017-11D,b'06848-20',BANCO DO BRASIL S/A CENTRO ...,BANCO DO BRASIL S/A CENTRO,623.80,623.80
3,31/07/2020,2020-07-31 00:00:00.000,9169,9169.0,9169.0,09169-20-PMFS,,b'09169-20',TRIBUNAL R.DO TRAB.DA 5.REGIAO ...,TRIBUNAL R.DO TRAB.DA 5.REGIAO,131788.71,131788.71
4,31/07/2020,2020-07-31 00:00:00.000,9485,9485.0,9485.0,09485-20-PMFS,,b'09485-20',BANCO DO BRASIL S/A CENTRO ...,BANCO DO BRASIL S/A CENTRO,3497.26,3497.26
...,...,...,...,...,...,...,...,...,...,...,...,...
1833,01/07/2020,2020-07-01 00:00:00.000,5742,5742.0,5742.0,05742-20-FMS,051-20161111,b'05742-20',REDESAUDE COOPERATIVA DE TRABALHO ...,REDESAUDE COOPERATIVA DE TRABALHO,245271.74,230964.23
1834,01/07/2020,2020-07-01 00:00:00.000,8717,8717.0,8717.0,08717-20-PMFS,,b'08717-20',MARYNES RODRIGUES DE MIRANDA ...,MARYNES RODRIGUES DE MIRANDA,-656.00,-656.00
1835,01/07/2020,2020-07-01 00:00:00.000,8715,8715.0,8715.0,08715-20-PMFS,,b'08715-20',EDNALVA VITORIO DE SOUZA ...,EDNALVA VITORIO DE SOUZA,-38.00,-38.00
1836,01/07/2020,2020-07-01 00:00:00.000,8888,8888.0,8888.0,08888-20-PMFS,,b'08888-20',BANCO DO BRASIL S/A CENTRO ...,BANCO DO BRASIL S/A CENTRO,5.50,5.50


In [146]:
pd.set_option('display.max_rows', None)

found_different_values = merged_df[merged_df["Valor"] != merged_df["valor"]][fields]
found_different_values.sort_values('Numero')

,Data de Publicacao,data,numero_tcmba,numero_prefeitura,Numero,NumeroDocumeto,N do processo,numeroProcesso,Credor,credor,Valor,valor
1831,01/07/2020,2020-07-01 00:00:00.000,5708,5708.0,5708.0,05708-20-FMS,069/20151111,b'05708-20',COOPERSADE COOP.S.P.ESP.SAUDE ...,COOPERSADE COOP.S.P.ESP.SAUDE,142433.71,128412.52
1810,01/07/2020,2020-07-01 00:00:00.000,5710,5710.0,5710.0,05710-20-FMS,069/20151111,b'05710-20',COOPERSADE COOP.S.P.ESP.SAUDE ...,COOPERSADE COOP.S.P.ESP.SAUDE,53666.97,48383.99
1796,01/07/2020,2020-07-01 00:00:00.000,5711,5711.0,5711.0,05711-20-FMS,069/20151111,b'05711-20',COOPERSADE COOP.S.P.ESP.SAUDE ...,COOPERSADE COOP.S.P.ESP.SAUDE,16503.59,14878.98
1814,01/07/2020,2020-07-01 00:00:00.000,5713,5713.0,5713.0,05713-20-FMS,001-2017-1111,b'05713-20',CLINICA SENHOR DO BONFIM ...,CLINICA SENHOR DO BONFIM,780128.83,741122.38
1816,01/07/2020,2020-07-01 00:00:00.000,5715,5715.0,5715.0,05715-20-FMS,001-2017-1111,b'05715-20',IUNE -INST. DE URO E NEFRO FSA ...,IUNE -INST. DE URO E NEFRO FSA,282656.51,268523.68
1798,01/07/2020,2020-07-01 00:00:00.000,5717,5717.0,5717.0,05717-20-FMS,100-2018,b'05717-20',SOFTWAREDATA SOL.P/ INF. LTDA ...,SOFTWAREDATA SOL.P/ INF. LTDA,88448.51,86679.54
1799,01/07/2020,2020-07-01 00:00:00.000,5718,5718.0,5718.0,05718-20-FMS,069/20151111,b'05718-20',COOPERSADE COOP.S.P.ESP.SAUDE ...,COOPERSADE COOP.S.P.ESP.SAUDE,559624.69,504535.18
1808,01/07/2020,2020-07-01 00:00:00.000,5721,5721.0,5721.0,05721-20-FMS,069/20151111,b'05721-20',COOPERSADE COOP.S.P.ESP.SAUDE ...,COOPERSADE COOP.S.P.ESP.SAUDE,8256.46,7443.69
1806,01/07/2020,2020-07-01 00:00:00.000,5723,5723.0,5723.0,05723-20-FMS,069/20151111,b'05723-20',COOPERSADE COOP.S.P.ESP.SAUDE ...,COOPERSADE COOP.S.P.ESP.SAUDE,8945.15,8064.58
1819,01/07/2020,2020-07-01 00:00:00.000,5725,5725.0,5725.0,05725-20-FMS,069/20151111,b'05725-20',COOPERSADE COOP.S.P.ESP.SAUDE ...,COOPERSADE COOP.S.P.ESP.SAUDE,98942.89,89202.94


In [147]:
#found_different_values.sort_values('Numero').to_csv("despesas-com-valores-diferentes-julho2020.csv")

In [150]:
found_different_values[found_different_values.duplicated(subset=['Numero'], keep=False)]

,Data de Publicacao,data,numero_tcmba,numero_prefeitura,Numero,NumeroDocumeto,N do processo,numeroProcesso,Credor,credor,Valor,valor
214,28/07/2020,2020-07-28 00:00:00.000,6733,6733.0,6733.0,06733-20-FMS,,b'06733-20',FOLHA PAG. ENDEMIAS ESTATUTARI ...,FOLHA PAG. ENDEMIAS ESTATUTARI,280969.50,8366.50
215,28/07/2020,2020-07-28 00:00:00.000,6733,6733.0,6733.0,06733-20-FMS,,b'06733-20',FOLHA PAG. ENDEMIAS ESTATUTARI ...,FOLHA PAG. ENDEMIAS ESTATUTARI,280969.50,8366.50
216,28/07/2020,2020-07-28 00:00:00.000,6732,6732.0,6732.0,06732-20-FMS,,b'06732-20',FOLHA DE PAGAMENTO SAMU ...,FOLHA DE PAGAMENTO SAMU,1737.39,1576.83
217,28/07/2020,2020-07-28 00:00:00.000,6732,6732.0,6732.0,06732-20-FMS,,b'06732-20',FOLHA DE PAGAMENTO SAMU ...,FOLHA DE PAGAMENTO SAMU,1737.39,1576.83
218,28/07/2020,2020-07-28 00:00:00.000,6739,6739.0,6739.0,06739-20-FMS,,b'06739-20',FOLHA DE PAGAMENTO SAMU ...,FOLHA DE PAGAMENTO SAMU,196082.78,162496.62
219,28/07/2020,2020-07-28 00:00:00.000,6739,6739.0,6739.0,06739-20-FMS,,b'06739-20',FOLHA DE PAGAMENTO SAMU ...,FOLHA DE PAGAMENTO SAMU,196082.78,162496.62
220,28/07/2020,2020-07-28 00:00:00.000,6740,6740.0,6740.0,06740-20-FMS,,b'06740-20',FOLHA DE PAGAMENTO SAMU ...,FOLHA DE PAGAMENTO SAMU,3128.26,2070.05
221,28/07/2020,2020-07-28 00:00:00.000,6740,6740.0,6740.0,06740-20-FMS,,b'06740-20',FOLHA DE PAGAMENTO SAMU ...,FOLHA DE PAGAMENTO SAMU,3128.26,2070.05
225,28/07/2020,2020-07-28 00:00:00.000,6730,6730.0,6730.0,06730-20-FMS,,b'06730-20',FOLHA PAGAMENTO ACS ESTATUTARIO ...,FOLHA PAGAMENTO ACS ESTATUTARIO,67071.67,39159.44
226,28/07/2020,2020-07-28 00:00:00.000,6730,6730.0,6730.0,06730-20-FMS,,b'06730-20',FOLHA PAGAMENTO ACS ESTATUTARIO ...,FOLHA PAGAMENTO ACS ESTATUTARIO,67071.67,39159.44
